In [27]:
import pandas as pd

In [30]:
#Uploading the data
xls = pd.ExcelFile('The_data.xlsx')

#Age and Sex
df_age_sex = pd.read_excel(xls, 'Age and Sex')
df_age_sex = df_age_sex.drop('YEAR', 1)


#elevation
df_elevation = pd.read_excel(xls, 'Elevation')
df_elevation['Elevation']=df_elevation['Elevation'].fillna("No Data")


#Marital Status
df_marital_status = pd.read_excel(xls, 'Marital Status')
df_marital_status = df_marital_status.drop('YEAR', 1)


#Educational Attainment
df_educational_attainment = pd.read_excel(xls, 'Educational Attainment')
df_educational_attainment = df_educational_attainment.drop('YEAR', 1)


#Types of Households
df_housing_units_type = pd.read_excel(xls, 'Housing Units Type')
df_housing_units_type = df_housing_units_type.drop('YEAR', 1)


#House Value
df_housing_value = pd.read_excel(xls, 'Housing Value')
df_housing_value = df_housing_value.drop('YEAR', 1)


#House Structure Age
df_year_house_built = pd.read_excel(xls, 'Year Structure Built')
df_year_house_built = df_year_house_built.drop('YEAR', 1)


#Vehicle Data
df_vehicles = pd.read_excel(xls, 'Vehicle Availability')
df_vehicles = df_vehicles.drop('YEAR', 1)

#Population Density
df_pop_density = pd.read_excel(xls, 'Population Density')



In [31]:
df_pop_density

,Neighborhood,People per square mile
0,National City,8110
1,Bostonia,7930
2,El Cajon,6950
3,Lemon Grove,6610
4,La Presa,6490
5,Imperial Beach,6400
6,La Mesa,6350
7,Vista,5090
8,Chula Vista,5000
9,Winter Gardens,4590
